<a href="https://colab.research.google.com/github/Vaibhavs10/optimise-my-whisper/blob/main/insanely_fast_whisper_fp16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Insanely Fast Whisper: A journey to build the fastest possible transcription with Whisper 🔥

By: [Vaibhav (VB) Srivastav](https://twitter.com/reach_vb)

## Base case: fp16

Efficient half-precision kernels for inference.

### Setup our inference environment 🧑‍💻

In [1]:
!pip install -q --upgrade transformers accelerate torch ipython-autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.3 MB/s eta 0:00:00


### Setting up the utilities to track time taken by each step ⏳

In [2]:
%load_ext autotime

time: 217 µs (started: 2024-05-24 06:45:22 +00:00)


### Necessary imports 🔧




In [3]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

time: 17.1 s (started: 2024-05-24 06:45:27 +00:00)


### Define Model checkpoint, device and datatype 🔉

In [4]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(128, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 1280)
      (layers): ModuleList(
        (0-31): 32 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
            (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1280, out_features=5120, bias=True)
          (fc2): Linear(in_features=5120, out_features=1280, bia

time: 19.6 s (started: 2024-05-24 06:45:55 +00:00)


### Load the model and initialise the speech recognition pipeline ⚡

In [5]:
processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    torch_dtype=torch_dtype,
    device=device,
)

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


time: 10.3 s (started: 2024-05-24 06:46:34 +00:00)


### Define an audio sample to test on 👇

10 minute audio to transcribe find it [here](https://huggingface.co/datasets/reach-vb/random-audios/resolve/main/4469669-10.mp3)

In [6]:
sample = "https://huggingface.co/datasets/reach-vb/random-audios/resolve/main/4469669-10.mp3"

time: 378 µs (started: 2024-05-24 06:48:04 +00:00)


### Transcribe away! 💪

In [7]:
result = pipe(sample)

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


time: 1min 2s (started: 2024-05-24 06:48:09 +00:00)


In [8]:
print(result["text"])

 Thank you. and investors will be accepted from 5.30 to 6 o'clock Japan time. Please be aware of that. Now, we will be collecting questions via telephone conferencing system. As is informed to you beforehand, the conference call system will require the pre-registration beforehand. Let me introduce the presenter today, President and CEO, Satoshi Tsunakawa. Corporate Senior Executive Vice President Mamoru Hatazawa. Representative Executive Officer, Corporate Executive Vice President, NCFO, Masayoshi Hirata. We have a chairperson of Strategic Review Committee Outside Director, Paul Brough. He is joining from Hong Kong online. My name is Hara of Corporate Communications Department. We are providing simultaneous translation, so if you are watching the live streaming in Japanese, you will be able to hear translation's voice. Please be aware of that. First, before going into transforming Toshiba to enhance Shihara's value, May I have Mr. Tanaka to say a few words upon the receipt of the repor